In [31]:
import urllib
import plotly.graph_objects as go
import numpy as np
import random

In [20]:
EX_GRAPH0 = {0: set([1, 2]),
            1: set([]),
            2: set([])}
EX_GRAPH1 = {0: set([1, 4, 5]),
            1: set([2, 6]),
            2: set([3]),
            3: set([0]),
            4: set([1]),
            5: set([2]),
            6: set([])}
EX_GRAPH2 = {0: set([1, 4, 5]),
            1: set([2, 6]),
            2: set([3, 7]),
            3: set([7]),
            4: set([1]),
            5: set([2]),
            6: set([]),
            7: set([3]),
            8: set([1, 2]),
            9: set([0, 3, 4, 5, 6, 7])}
CITATION_URL = "http://storage.googleapis.com/codeskulptor-alg/alg_phys-cite.txt"

In [32]:
def make_complete_graph(num_nodes):
    complete_graph = {}
    if num_nodes <= 0:
        return complete_graph
    else:
        for node in range(num_nodes):
            node_list = set(range(num_nodes))
            node_list.remove(node)
            complete_graph[node] = node_list
        return complete_graph
    
def compute_in_degrees(digraph):
    in_degrees = {k: 0 for k in digraph.keys()}
    for node in digraph.keys():
        for head in digraph[node]:
            in_degrees[head] += 1
    return in_degrees

def in_degree_distribution(digraph):
    in_degrees_dict = compute_in_degrees(digraph)
    in_degrees_dist = {}
    in_degrees_list = list(in_degrees_dict.values())
    for in_degree in in_degrees_list:
        in_degrees_dist[in_degree] = in_degrees_list.count(in_degree)
    return in_degrees_dist

def load_graph(graph_url):
    """
    Function that loads a graph given the URL
    for a text representation of the graph
    
    Returns a dictionary that models a graph
    """
    graph_file = urllib.request.urlopen(graph_url)
    graph_text = graph_file.read().decode('utf-8')  # Decode bytes to string
    graph_lines = graph_text.split('\n')
    graph_lines = graph_lines[:-1]  # If needed, remove the last empty line

    print(f"Loaded graph with {len(graph_lines)} nodes")

    
    answer_graph = {}
    for line in graph_lines:
        neighbors = line.split(' ')
        node = int(neighbors[0])
        answer_graph[node] = set([])
        for neighbor in neighbors[1 : -1]:
            answer_graph[node].add(int(neighbor))

    return answer_graph

def normalise_in_degree_dist(dist):
    num_nodes = sum(dist.values())
    norm_in_degree_dist = {k: float(v) / num_nodes for k, v in dist.items()}
    
    return norm_in_degree_dist

def plot_log_log_graph(data):

    # Extract keys and values
    keys = list(data.keys())
    values = list(data.values())

    # Remove zero keys and corresponding values to avoid log(0) issues
    filtered_keys = [key for key in keys if key > 0]
    filtered_values = [data[key] for key in filtered_keys]

    # Apply log to keys and values
    log_keys = np.log(filtered_keys)
    log_values = np.log(filtered_values)

    # Create a scatter plot with Plotly
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=log_keys,
        y=log_values,
        mode='markers',
        marker=dict(size=10, color='blue'),
        name='Log-Log Plot'
    ))

    # Update layout
    fig.update_layout(
        title='Log-Log Plot of number of citations vs. norm_distro',
        xaxis_title='Log of number of citations',
        yaxis_title='Log of norm_distro',
        xaxis=dict(type='linear'),
        yaxis=dict(type='linear'),
        showlegend=True
    )

    # Show the plot
    fig.show()

def dpa_algo(connect_to_m_nodes, final_n_nodes):
    """
    Generate a directed graph using the DPA (Directed Preferential Attachment) algorithm.
    
    This algorithm starts with a complete graph of `connect_to_m_nodes` nodes.
    It then adds new nodes to the graph, one at a time, each connecting to 
    `connect_to_m_nodes` existing nodes selected with probability proportional 
    to their current in-degrees.

    Parameters:
    connect_to_m_nodes (int): The number of nodes to connect to initially.
                              This is also the number of neighbors each new node will connect to.
    final_n_nodes (int): The total number of nodes in the final graph.

    Returns:
    dict: A dictionary representing the adjacency list of the directed graph.
          Keys are node indices, and values are sets of nodes that the key node has directed edges to.
    
    """
    m_node_complete_graph = make_complete_graph(connect_to_m_nodes)
    
    dpatrial = DPATrial(connect_to_m_nodes)

    for i in range(connect_to_m_nodes, final_n_nodes):
        # tot_in_deg = sum(compute_in_degrees(m_node_complete_graph).values())
                
        nodes_to_add = dpatrial.run_trial(connect_to_m_nodes)    
        
        m_node_complete_graph[i] = nodes_to_add

    return m_node_complete_graph

class DPATrial:
    """
    Simple class to encapsulate optimized trials for DPA algorithm
    
    Maintains a list of node numbers with multiple instances of each number.
    The number of instances of each node number are
    in the same proportion as the desired probabilities
    
    Uses random.choice() to select a node number from this list for each trial.
    """

    def __init__(self, num_nodes):
        """
        Initialize a DPATrial object corresponding to a 
        complete graph with num_nodes nodes
        
        Note the initial list of node numbers has num_nodes copies of
        each node number
        """
        self._num_nodes = num_nodes
        self._node_numbers = [node for node in range(num_nodes) for dummy_idx in range(num_nodes)]


    def run_trial(self, num_nodes):
        """
        Conduct num_node trials using by applying random.choice()
        to the list of node numbers
        
        Updates the list of node numbers so that the number of instances of
        each node number is in the same ratio as the desired probabilities
        
        Returns:
        Set of nodes
        """
        
        # compute the neighbors for the newly-created node
        new_node_neighbors = set()
        for dummy_idx in range(num_nodes):
            new_node_neighbors.add(random.choice(self._node_numbers))
        
        # update the list of node numbers so that each node number 
        # appears in the correct ratio
        self._node_numbers.append(self._num_nodes)
        self._node_numbers.extend(list(new_node_neighbors))
        
        #update the number of nodes
        self._num_nodes += 1
        return new_node_neighbors

In [22]:
citation_graph = load_graph(CITATION_URL)

Loaded graph with 27770 nodes


In [26]:
len(citation_graph.keys())

27770

In [27]:
count_edges = sum(len(v) for v in citation_graph.values())
print(f"Loaded graph has {count_edges} edges")

avg_out_deg = count_edges / len(citation_graph.keys())
print(f"Loaded graph has avg {avg_out_deg} avg out degree")

print(f"Use {len(citation_graph.keys())} as n and {avg_out_deg} as m")

Loaded graph has 352768 edges
Loaded graph has avg 12.703204897371265 avg out degree
Use 27770 as n and 12.703204897371265 as m


In [28]:
norm_dist = normalise_in_degree_dist(in_degree_distribution(citation_graph))
plot_log_log_graph(norm_dist)

In [83]:
dpa_graph = dpa_algo(13, 28000)
dpa_graph

{0: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12},
 1: {0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12},
 2: {0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12},
 3: {0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12},
 4: {0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12},
 5: {0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12},
 6: {0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12},
 7: {0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12},
 8: {0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12},
 9: {0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12},
 10: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12},
 11: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12},
 12: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11},
 13: {0, 2, 5, 8, 9, 10, 12},
 14: {0, 2, 3, 4, 5, 6, 7, 9, 10, 12},
 15: {0, 1, 3, 5, 6, 7, 8},
 16: {0, 1, 2, 3, 5, 6, 8, 10, 12},
 17: {0, 1, 2, 3, 4, 5, 7, 8, 9},
 18: {0, 2, 4, 5, 6, 7, 9, 10, 11, 12},
 19: {0, 2, 5, 7, 8, 9, 11, 12},
 20: {2, 4, 6, 7, 8, 10, 11, 12},
 21: {0, 3, 4, 5, 6, 7, 9, 10, 12},
 22: {3, 4, 5, 8, 9, 10, 11, 16},
 23: {1, 2, 3, 5, 7, 9, 10, 11, 12},
 24: {1, 2, 5, 9, 10, 11, 12},
 25

In [85]:
norm_dist2 = normalise_in_degree_dist(in_degree_distribution(dpa_graph))
plot_log_log_graph(norm_dist2)
fig.update_layout(
        title='Log-Log Plot of in degree distribution for DPA graph',
        xaxis_title='Log of number of edges',
        yaxis_title='Log of norm_distro'
    )

NameError: name 'fig' is not defined